In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Question 1(a)

In [127]:
P_hostel = 0.60
P_dayScholar = 0.40
P_A_hostel = 0.30
P_A_dayScholar = 0.20

In [128]:
P_hostel_A = P_hostel * P_A_hostel / (P_hostel*P_A_hostel + P_dayScholar * P_A_dayScholar)
print(f"Probabilty of student being hosteler if has A grade : {P_hostel_A}")

Probabilty of student being hosteler if has A grade : 0.6923076923076923


### Question 1(b)

In [129]:
P_disease = 0.01
P_pos_disease = 0.99
P_pos_noDisease = 0.02

In [130]:
P_noDisease = 1 - P_disease

In [131]:
P_disease_pos = P_disease * P_pos_disease / (P_disease*P_pos_disease + P_noDisease*P_pos_noDisease)
print(f"Probabilty of having disease if positive test result : {P_disease_pos}")

Probabilty of having disease if positive test result : 0.3333333333333333


### Question 2

In [132]:
df = pd.read_csv("w7q2.csv")
df

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31…40,high,no,fair,no
3,>40,medium,no,fair,no
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31…40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [133]:
from collections import defaultdict
def naive_bayes_train(df):
    priors = df['buys_computer'].value_counts(normalize=True).to_dict()
    features = df.columns[:-1]
    conditional_probs = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    
    for feature in features:
        for value in df[feature].unique():
            for outcome in df['buys_computer'].unique():
                subset = df[df['buys_computer'] == outcome]
                prob = (subset[feature] == value).sum() / len(subset)
                conditional_probs[feature][value][outcome] = prob
                
    return priors, conditional_probs

def naive_bayes_predict(priors, conditional_probs, sample):
    posteriors = dict.fromkeys(priors.keys(), 1)
    for outcome in priors:
        posteriors[outcome] = np.log(priors[outcome])
        for feature, value in sample.items():
            prob = conditional_probs[feature][value].get(outcome, 1e-10)
            posteriors[outcome] += np.log(prob)
    return max(posteriors, key=posteriors.get)

priors, conditional_probs = naive_bayes_train(df)
sample = {'age': '31…40', 'income': 'medium', 'student': 'no', 'credit_rating': 'excellent'}
prediction = naive_bayes_predict(priors, conditional_probs, sample)
print(f'The predicted class for the sample is: {prediction}')

The predicted class for the sample is: no


### Question 3

In [134]:
df = pd.read_csv('w7q3.csv')
df

,text,tag
0,A great game,Sports
1,The election was over,Not sports
2,Very clean match,Sports
3,A clean but forgettable game,Sports
4,It was a close election,Not sports


In [135]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.model_selection import train_test_split

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text) 
    text = re.sub(r'\s+', ' ', text) 
    return text

df['text'] = df['text'].apply(preprocess_text)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def train_naive_bayes(train_df):
    class_counts = train_df['tag'].value_counts().to_dict()
    total_docs = len(train_df)
    priors = {tag: count / total_docs for tag, count in class_counts.items()}
    
    word_counts = {tag: Counter() for tag in class_counts.keys()}
    class_word_counts = {tag: 0 for tag in class_counts.keys()}
    
    for _, row in train_df.iterrows():
        words = row['text'].split()
        tag = row['tag']
        class_word_counts[tag] += len(words)
        for word in words:
            word_counts[tag][word] += 1
    vocab = set(word for words in word_counts.values() for word in words)

    conditional_probs = {tag: {} for tag in class_counts.keys()}
    
    for tag in class_counts.keys():
        total_words = class_word_counts[tag]
        for word in vocab:
            conditional_probs[tag][word] = (word_counts[tag].get(word, 0) + 1) / (total_words + len(vocab))
    
    return priors, conditional_probs, vocab, class_word_counts

def predict_naive_bayes(text, priors, conditional_probs, vocab, class_word_counts):
    words = text.split()
    posteriors = {tag: np.log(prior) for tag, prior in priors.items()}
    
    for tag in priors:
        for word in words:
            prob = conditional_probs[tag].get(word, 1 / (class_word_counts[tag] + len(vocab)))
            posteriors[tag] += np.log(prob)
    
    return max(posteriors, key=posteriors.get)

priors, conditional_probs, vocab, class_word_counts = train_naive_bayes(train_df)

sample_text = "A very close game"
sample_text_processed = preprocess_text(sample_text)
prediction = predict_naive_bayes(sample_text_processed, priors, conditional_probs, vocab, class_word_counts)
print(f'The predicted class for the sample text "{sample_text}" is: {prediction}')

The predicted class for the sample text "A very close game" is: Sports
